In [ ]:
#!pip install transformers[torch] datasets torch huggingface_hub numpy evaluate librosa soundfile
#!pip install numexpr -U

In [ ]:
from datasets import load_dataset
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, TrainingArguments, Trainer
import torch
from huggingface_hub import notebook_login
import numpy as np
import evaluate

In [ ]:
gtzan = load_dataset("marsyas/gtzan", "all")
gtzan

In [ ]:
gtzan = gtzan["train"].train_test_split(seed=42, shuffle=True, test_size=0.2)
id2label_fn = gtzan["train"].features["genre"].int2str

In [ ]:
max_duration = 30.0

model_id = "ntu-spml/distilhubert"
feature_extractor = AutoFeatureExtractor.from_pretrained(
    model_id, do_normalize=True, return_attention_mask=True
)

def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
        return_attention_mask=True,
    )
    return inputs

In [ ]:
gtzan_encoded = gtzan.map(
    preprocess_function,
    remove_columns=["audio", "file"],
    batched=True,
    batch_size=100,
    num_proc=1,
)

In [ ]:
gtzan_encoded = gtzan_encoded.rename_column("genre", "label")

id2label = {
    str(i): id2label_fn(i)
    for i in range(len(gtzan_encoded["train"].features["label"].names))
}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
num_labels = len(id2label)

model = AutoModelForAudioClassification.from_pretrained(
    model_id,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

In [ ]:
notebook_login()

In [ ]:
model_name = model_id.split("/")[-1]

In [ ]:
from transformers import EarlyStoppingCallback

early_stop_callback = EarlyStoppingCallback(early_stopping_patience  = 3)

In [ ]:
## use this training_args to train
training_args = TrainingArguments(
    f"{model_name}-finetuned-gtzan",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=4e-5,
    warmup_ratio=0.1,
    num_train_epochs=100,             # total number of training epochs
    per_device_train_batch_size=8,    # batch size per device during training
    per_device_eval_batch_size=8,     # batch size for evaluation
    warmup_steps=500,                 # number of warmup steps for learning rate scheduler
    weight_decay=0.1,                 # strength of weight decay, optimal is 0.1
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=False,
    push_to_hub=True,
    gradient_accumulation_steps=1,
)

In [ ]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
gtzan_encoded["train"].set_format(type='torch')
gtzan_encoded["test"].set_format(type='torch')

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=gtzan_encoded["train"],
    eval_dataset=gtzan_encoded["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    callbacks = [early_stop_callback],
)

In [ ]:
trainer.train()

In [ ]:
# push results to Huggingface
kwargs = {
    "dataset_tags": "marsyas/gtzan",
    "dataset": "GTZAN",
    "model_name": f"{model_name}-finetuned-gtzan",
    "finetuned_from": model_id,
    "tasks": "audio-classification",
}

trainer.push_to_hub(**kwargs)